# Create tables from Master SQL Database

### Import Dependencies

In [14]:
import os
import pandas as pd
import plotly.graph_objects as go
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import text

In [15]:
# Define the destination folder and database path
destination_folder = 'data'
database_filename = 'master.sqlite'
destination_path = os.path.join(destination_folder, database_filename)

# Create an SQLite engine
engine = create_engine(f'sqlite:///{destination_path}')

In [16]:
## Analysis

In [23]:
# Query to drop the table if it exists
query_drop = "DROP TABLE IF EXISTS rl_modified;"

# Execute
with engine.connect() as conn:
    conn.execute(text(query_drop))

# Create the modified table
query_multiply = """
CREATE TABLE rl_modified AS
SELECT
    Area,
    Year,
    "Land area" * 1000 AS Land_area,
    Cropland * 1000 AS Cropland,
    "Permanent meadows and pastures" * 1000 AS Permanent_meadows_and_pastures,
    "Forest land" * 1000 AS Forest_land,
    "Other land" * 1000 AS Other_land,
    "Farm buildings and Farmyards" * 1000 AS Farm_buildings_and_Farmyards,
    "Unaccounted" * 1000 AS Unaccounted
FROM rl;
"""

# Execute
with engine.connect() as conn:
    conn.execute(text(query_multiply))

In [34]:
# Define SQL query
query_percentage = """
WITH Crop_Percentage AS (
    SELECT
        qcl.Area,
        qcl.Item,
        qcl.Year,
        rl_m.Land_area,  -- Ensure you're referencing the correct field
        qcl.Value AS Crop_Value,
        (qcl.Value / rl_m.Land_area) * 100 AS Agricultural_Land_Percentage  -- Calculate percentage of total land
    FROM qcl
    JOIN rl_modified rl_m ON qcl.Area = rl_m.Area  -- Use rl_m.Area instead of tl.Area
    WHERE qcl.Year = 2022
)
SELECT * 
FROM Crop_Percentage
ORDER BY Agricultural_Land_Percentage DESC;
"""

crop_percentage_2022 = pd.read_sql(query_percentage, con=engine)

print(crop_percentage_2022)
crop_percentage_2022.describe()

              Area                  Item  Year   Land_area  Crop_Value  \
0       Bangladesh  Cattle and Buffaloes  2022  13017000.0    26208000   
1       Bangladesh  Cattle and Buffaloes  2022  13017000.0    26208000   
2       Bangladesh  Cattle and Buffaloes  2022  13017000.0    26208000   
3       Bangladesh  Cattle and Buffaloes  2022  13017000.0    26208000   
4       Bangladesh  Cattle and Buffaloes  2022  13017000.0    26208000   
...            ...                   ...   ...         ...         ...   
46765  Saint Lucia      Green corn/Maize  2022     61000.0           0   
46766  Saint Lucia      Green corn/Maize  2022     61000.0           0   
46767  Saint Lucia      Green corn/Maize  2022     61000.0           0   
46768  Saint Lucia      Green corn/Maize  2022     61000.0           0   
46769  Saint Lucia      Green corn/Maize  2022     61000.0           0   

       Agricultural_Land_Percentage  
0                        201.336714  
1                        201.336714

,Year,Land_area,Crop_Value,Agricultural_Land_Percentage
count,46770.0,4.677000e+04,4.677000e+04,46770.000000
mean,2022.0,7.316808e+07,2.911979e+06,5.639142
std,0.0,1.792343e+08,1.613007e+07,14.924375
min,2022.0,2.000000e+03,0.000000e+00,0.000000
25%,2022.0,3.541000e+06,3.263000e+03,0.041238
50%,2022.0,1.925300e+07,6.395600e+04,0.476092
75%,2022.0,5.818000e+07,7.611110e+05,3.974752
max,2022.0,1.638995e+09,3.054632e+08,201.336714


In [35]:
# Define SQL query
query_percentage = """
WITH Crop_Percentage AS (
    SELECT
        qcl.Area AS Country,
        qcl.Item,
        qcl.Year,
        rl_m.Land_area,
        qcl.Value AS Crop_Value,
        (qcl.Value / rl_m.Land_Area) * 100 AS Agricultural_Land_Percentage
    FROM qcl
    JOIN rl_modified rl_m ON qcl.Area = rl_m.Area
    WHERE qcl.Year = 2022
)
SELECT DISTINCT Country
FROM Crop_Percentage
WHERE Agricultural_Land_Percentage > 100
ORDER BY Country;
"""

crop_percentage_countries_outlier_2022 = pd.read_sql(query_percentage, con=engine)

print(crop_percentage_countries_outlier_2022)

                        Country
0                    Bangladesh
1                         India
2  Netherlands (Kingdom of the)
3                      Pakistan
